In [1]:
import pandas as pd
import mysql.connector
from dotenv import load_dotenv
import os

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
load_dotenv()

True

In [6]:
# Establish a connection

try:
    conn = mysql.connector.connect(
        host= 'localhost',
        user=os.getenv('db_user'),
        password=os.getenv('db_pass'),
        database='swiggy'
    )
    print('Connection established')
except mysql.connector.Error as e:
    print(f'Error: {e}')

Connection established


In [11]:
# Create a cursor

cursor = conn.cursor()

In [14]:
swiggy = pd.read_sql_query('show tables;', conn)
swiggy

,Tables_in_swiggy
0,delivery_partner
1,food
2,menu
3,order_details
4,orders
5,restaurants
6,users


### Clean the 'orders' table

In [15]:
cursor.execute('''alter table orders
modify column date date;''')

In [16]:
desc = pd.read_sql_query('desc orders;',conn)
desc

,Field,Type,Null,Key,Default,Extra
0,order_id,b'int',YES,,None,
1,user_id,b'int',YES,,None,
2,r_id,b'int',YES,,None,
3,amount,b'int',YES,,None,
4,date,b'date',YES,,None,
5,partner_id,b'int',YES,,None,
6,delivery_time,b'int',YES,,None,
7,delivery_rating,b'int',YES,,None,
8,restaurant_rating,b'text',YES,,None,


## Qustion 1: Find customers who have never ordered.

In [20]:
not_orderd_users = pd.read_sql_query('''select user_id, name
from users
where user_id not in (select user_id from orders);''', conn)

not_orderd_users

,user_id,name
0,6,Anupama
1,7,Rishabh


## Question 2: Average price per dish.

In [19]:
avg_price_dish = pd.read_sql_query('''select a.f_name, round(avg(b.price)) as Avg_price
from food a
inner join menu b on a.f_id = b.f_id
group by a.f_name;''', conn)

avg_price_dish

,f_name,Avg_price
0,Non-veg Pizza,450.0
1,Veg Pizza,400.0
2,Choco Lava cake,98.0
3,Chicken Wings,230.0
4,Chicken Popcorn,300.0
5,Rice Meal,213.0
6,Roti meal,140.0
7,Masala Dosa,180.0
8,Rava Idli,120.0
9,Schezwan Noodles,220.0


## Question 3: Find top restaurants in terms of number of orders for a given month

In [21]:
top_rest_month_orders = pd.read_sql_query('''select a.r_name as 'Restaurant', month(b.date) as 'Month', count(b.order_id) as 'Total Orders'
from restaurants a
inner join orders b on a.r_id = b.r_id
group by a.r_name, month(b.date)
order by month(b.date), 'Total Orders';''', conn)

top_rest_month_orders

,Restaurant,Month,Total Orders
0,dominos,5,2
1,kfc,5,2
2,Dosa Plaza,5,3
3,box8,6,2
4,dominos,6,1
5,kfc,6,3
6,Dosa Plaza,6,1
7,China Town,6,1
8,box8,7,2
9,Dosa Plaza,7,1


## Question 4: Restaurants with monthly sales more than x for

##### For this question let's consider x=500 (i.e. Restaurants with monthly sales more than 500)

In [23]:
rest_monthly_sale_x = pd.read_sql_query('''select a.r_name as 'Restaurant', month(b.date) as 'Month', sum(b.amount) as Revenue
from restaurants a
inner join orders b on a.r_id = b.r_id
group by a.r_name, month(b.date)
having Revenue>500
order by month(b.date), Revenue desc;''', conn)

rest_monthly_sale_x

,Restaurant,Month,Revenue
0,dominos,5,1000.0
1,Dosa Plaza,5,780.0
2,kfc,5,645.0
3,kfc,6,990.0
4,dominos,6,950.0
5,kfc,7,1935.0
6,dominos,7,1100.0
7,China Town,7,1050.0


## Question 5: Show all orders with order details for a particular customer in a particular date range.

##### For this question let's pull out the order details for 'Nitish' between 10 June 2022 and 10 July 2022.

In [24]:
order_details_user_daterange =pd.read_sql_query('''select a.name, b.date, c.r_name as 'Restaurant', e.f_name as 'Food name', count(b.order_id) as Total_orders, sum(b.amount) as Money_spent
from users a
inner join orders b on a.user_id = b.user_id
inner join restaurants c on b.r_id = c.r_id
inner join order_details d on b.order_id = d.order_id
inner join food e on d.f_id = e.f_id
where a.name like '%Nitish%' and b.date between '2022-06-10' and '2022-07-10'
group by a.name, b.date, c.r_name, e.f_name
order by b.date;''', conn)

order_details_user_daterange

,name,date,Restaurant,Food name,Total_orders,Money_spent
0,Nitish,2022-06-15,box8,Choco Lava cake,1,240.0
1,Nitish,2022-06-15,box8,Rice Meal,1,240.0
2,Nitish,2022-06-29,box8,Choco Lava cake,1,240.0
3,Nitish,2022-06-29,box8,Rice Meal,1,240.0
4,Nitish,2022-07-10,box8,Choco Lava cake,1,220.0
5,Nitish,2022-07-10,box8,Roti meal,1,220.0


## Question 6: Find restaurant with maximum repeated customers.

In [29]:
repeated_customers = pd.read_sql_query('''select d.r_name as 'Restaurant', count(c.Visits) as Repeated_customers from
(select r_id, user_id, count(order_id) as Visits
from orders
group by r_id, user_id
having Visits> 1) c
inner join restaurants d on c.r_id = d.r_id
group by d.r_name
order by Repeated_customers desc
limit 1;''', conn)

repeated_customers

,Restaurant,Repeated_customers
0,kfc,2


## Question 7:  Month over month revenue growth of Swiggy.

In [26]:
MoM_revenue = pd.read_sql_query('''select c.*, c.Revenue - c.prv_month_rev as 'MoM_Revenue' from
(select month(date) as 'Month', sum(amount) as Revenue, lag(sum(amount), 1) over() as prv_month_rev
from orders
group by month(date)
order by month(date)) c;
''', conn)

MoM_revenue

,Month,Revenue,prv_month_rev,MoM_Revenue
0,5,2425.0,NaN,NaN
1,6,3220.0,2425.0,795.0
2,7,4845.0,3220.0,1625.0


## Question 8:  Favorite food of each customer.

In [27]:
fav_food_customer = pd.read_sql_query('''select e.user_id, e.name, e.`Food name`, e.Total_orders from
(select a.user_id, b.name, d.f_name as 'Food name', count(a.order_id) as Total_orders, dense_rank() over(partition by a.user_id order by count(a.order_id) desc) as rnk
from orders a
inner join users b on a.user_id = b.user_id
inner join order_details c on a.order_id = c.order_id
inner join food d on c.f_id = d.f_id
group by a.user_id, b.name, d.f_name
order by a.user_id, Total_orders desc) e
where e.rnk = 1;''', conn)

fav_food_customer

,user_id,name,Food name,Total_orders
0,1,Nitish,Choco Lava cake,5
1,2,Khushboo,Choco Lava cake,3
2,3,Vartika,Chicken Wings,3
3,4,Ankit,Schezwan Noodles,3
4,4,Ankit,Veg Manchurian,3
5,5,Neha,Choco Lava cake,5


## Question 9: Find most loyal customers for all restaurants.

In [30]:
loyal_customers = pd.read_sql_query('''select c.Restaurant, count(Visits) as Loyal_customers from
(select a.user_id, a.r_id, b.r_name as 'Restaurant', count(a.order_id) as Visits
from orders a
inner join restaurants b on a.r_id = b.r_id
group by a.user_id, a.r_id, b.r_name
having count(a.order_id) > 1
order by a.user_id, Visits desc) c
group by c.Restaurant
order by Loyal_customers desc;''', conn)

loyal_customers

,Restaurant,Loyal_customers
0,kfc,2
1,box8,1
2,Dosa Plaza,1
3,China Town,1
4,dominos,1


## Question 10: Month over month revenue growth of each restaurant.

In [31]:
MoM_revenue_per = pd.read_sql_query('''select c.*, concat(round(((c.Revenue - c.prv_month_rev)/c.prv_month_rev)*100, 2),'%') as 'MoM_growth' from
(select month(date) as 'Month', sum(amount) as Revenue, lag(sum(amount), 1) over() as prv_month_rev
from orders
group by month(date)
order by month(date)) c;''', conn)

MoM_revenue_per

,Month,Revenue,prv_month_rev,MoM_growth
0,5,2425.0,NaN,None
1,6,3220.0,2425.0,32.78%
2,7,4845.0,3220.0,50.47%


## Question 11: Find the most paired products.

In [32]:
paired_products = pd.read_sql_query('''select f1.f_name as product1, f2.f_name as product2, count(*) as pair_counts
from order_details od1
inner join order_details od2 on od1.order_id = od2.order_id and od1.f_id < od2.f_id
inner join food f1 on od1.f_id = f1.f_id
inner join food f2 on od2.f_id = f2.f_id
group by f1.f_name, f2.f_name
order by pair_counts desc
limit 1;''', conn)

paired_products

,product1,product2,pair_counts
0,Choco Lava cake,Chicken Wings,5


In [34]:
# Close the cursor and connection.
try:
    cursor.close()
    print('Cursor closed')
    conn.close()
    print('Connection closed')
except Exception as e:
    print(f'Error {e}')

Cursor closed
Connection closed
